In [ ]:

import glob
files = glob.glob('/content/drive/MyDrive/dataset/firewall-malware-analysis/*.csv')
df = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)


In [ ]:
print(df.columns)

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std',
       ...
       'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
       'Active Std', 'Active Max', 'Active Min', 'Idle Std', 'Idle Max',
       'Idle Min', 'Label'],
      dtype='object', length=144)


In [ ]:
print(df[' Label'].unique())

['BENIGN' 'DDoS' 'PortScan' 'Bot' 'Infiltration'
 'Web Attack � Brute Force' 'Web Attack � XSS'
 'Web Attack � Sql Injection' 'FTP-Patator' 'SSH-Patator' 'DoS slowloris'
 'DoS Slowhttptest' 'DoS Hulk' 'DoS GoldenEye' 'Heartbleed' nan]


In [ ]:
df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Std,Active Max,Active Min,Idle Std,Idle Max,Idle Min,Label
0,54865.0,3.0,2.0,0.0,12,0.0,6.0,6.0,6.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,55054.0,109.0,1.0,1.0,6,6.0,6.0,6.0,6.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,55055.0,52.0,1.0,1.0,6,6.0,6.0,6.0,6.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,46236.0,34.0,1.0,1.0,6,6.0,6.0,6.0,6.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,54863.0,3.0,2.0,0.0,12,0.0,6.0,6.0,6.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2880738,NaN,NaN,NaN,NaN,26,NaN,NaN,NaN,NaN,NaN,...,229.0,2.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2880739,NaN,NaN,NaN,NaN,76,NaN,NaN,NaN,NaN,NaN,...,-1.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2880740,NaN,NaN,NaN,NaN,2664,NaN,NaN,NaN,NaN,NaN,...,243.0,24.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2880741,NaN,NaN,NaN,NaN,26,NaN,NaN,NaN,NaN,NaN,...,229.0,2.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
df.columns = df.columns.str.strip()
df["Label"] = df["Label"].str.replace("�", "-", regex=False)


AttributeError: 'DataFrame' object has no attribute 'str'

In [ ]:
df.columns

In [ ]:
df["Label"] = (df["Label"] != "BENIGN").astype(int)

In [ ]:
attack_df = df[df["Label"] == 1]

attack_sample = (
    attack_df.sample(n=25000, random_state=42)
    if len(attack_df) > 25000
    else attack_df
)



In [ ]:
benign_df = df[df["Label"] == 0]

benign_sample = benign_df.sample(
    n=len(attack_sample),
    random_state=42
)


In [ ]:
df_minimization = (
    pd.concat([attack_sample, benign_sample])
      .sample(frac=1, random_state=42)
      .reset_index(drop=True)
)

df_minimization.to_csv('/content/drive/MyDrive/dataset/firewall-malware-analysis/df_minimization.csv', index=False)


In [3]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/dataset/firewall-malware-analysis/df_minimization.csv')

In [5]:
exception_col = 'Label'

for col in df.columns:
  if col != exception_col:
    df[col] = df[col].astype('float32')

df[exception_col] = df[exception_col].astype("int8")



In [6]:
DROP_COLS = [
    "Flow ID",
    "Source IP",
    "Source Port",
    "Destination IP",
    "Destination Port",
    "Timestamp",
    "Fwd Header Length.1"
]

df.drop(columns=DROP_COLS, inplace=True, errors="ignore")


In [7]:
X = df.drop("Label", axis=1).values
y = df["Label"].values

X = X.reshape(X.shape[0], X.shape[1], 1)


In [8]:
import numpy as np
print("The Shape of the Input data : " + str(X.shape))
print("The Shape of the Output data : " + str(y.shape))

The Shape of the Input data : (50000, 76, 1)
The Shape of the Output data : (50000,)


In [9]:
class Relu:
  def forward(self, input_data):
    return np.maximum(0, input_data)

  def backward(self, grad_out):
        return grad_out * self.mask

In [10]:
import numpy as np

class Conv1D:
  def __init__(self, kernel_size):
      self.kernel_size = kernel_size
      self.weights = np.random.randn(kernel_size)
      self.bias = np.random.randn()


  def forward(self, input_data):
    n = len(input_data)
    output_length = n - self.kernel_size + 1
    output = np.zeros(output_length)

    for i in range(output_length):
        window = input_data[i:i+self.kernel_size]
        output[i] = np.sum(window * self.weights) + self.bias

    return output

In [31]:
class Conv1DLayer:
    def __init__(self, num_filters, kernel_size):
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.filters = None

    def forward(self, x):
        batch, time, features = x.shape

        if self.filters is None:
            self.filters = np.random.randn(
                self.num_filters, self.kernel_size, features
            ) * 0.01

        output_time = time - self.kernel_size + 1
        out = np.zeros((batch, output_time, self.num_filters))

        for b in range(batch):
            for f in range(self.num_filters):
                for t in range(output_time):
                    window = x[b, t:t+self.kernel_size, :]
                    out[b, t, f] = np.sum(window * self.filters[f])

        return out


In [32]:
class MaxPooling1D:
  def __init__(self, pool_size):
    self.pool_size = pool_size

  def forward(self, input_data):
    pooled = []
    for i in range(0,len(input_data),self.pool_size):
      pooled.append(np.max(input_data[i:i+self.pool_size]))
    return np.array(pooled)

In [33]:
class MaxPooling1DLayer:
    def __init__(self, pool_size):
        self.pool_size = pool_size

    def forward(self, x):
        batch, time, features = x.shape
        output_time = time // self.pool_size

        out = np.zeros((batch, output_time, features))

        for b in range(batch):
            for t in range(output_time):
                window = x[b,
                           t*self.pool_size:(t+1)*self.pool_size,
                           :]
                out[b, t, :] = np.max(window, axis=0)

        return out


In [34]:
import numpy as np

class Dense:
    def __init__(self, input_dim, output_dim, activation=None):
        self.W = np.random.randn(input_dim, output_dim) * 0.01
        self.b = np.zeros((1, output_dim))
        self.activation = activation

    def forward(self, x):
        self.x = x
        z = np.dot(x, self.W) + self.b

        if self.activation == "relu":
            self.out = np.maximum(0, z)

        elif self.activation == "softmax":
            exp = np.exp(z - np.max(z, axis=1, keepdims=True))
            self.out = exp / np.sum(exp, axis=1, keepdims=True)

        elif self.activation == "sigmoid":
            self.out = 1 / (1 + np.exp(-z))

        else:
            self.out = z

        return self.out



In [38]:
conv1 = Conv1DLayer(num_filters=8, kernel_size=3)
x = conv1.forward(X)

pool1 = MaxPooling1DLayer(pool_size=2)
x = pool1.forward(x)

print("After first pooling:", x.shape)



conv2 = Conv1DLayer(num_filters=16, kernel_size=3)
x = conv2.forward(x)

pool2 = MaxPooling1DLayer(pool_size=2)
x = pool2.forward(x)

print("After second pooling:", x.shape)

batch, time, features = x.shape
x = x.reshape(batch, time * features)

dense = Dense(input_dim=time * features, output_dim=2, activation="softmax")
predictions = dense.forward(x)


After first pooling: (50000, 37, 8)
After second pooling: (50000, 17, 16)


In [1]:
predictions

NameError: name 'predictions' is not defined